In [1]:
# Install necessary libraries (if not installed)
!pip install transformers torch sentencepiece rouge-score nltk scikit-learn tqdm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.8 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=0d6c1c2798dfbc67ae38dbd59209950dde4790090b7c5f455d75d8f2a5c1390b
  S

In [2]:


import pandas as pd
import numpy as np
import torch
import zipfile
from transformers import BartTokenizer, BartForConditionalGeneration
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from tqdm import tqdm
from google.colab import files

# Mount Google Drive to access the model
from google.colab import drive
drive.mount('/content/drive')

# Path to the zip file in Google Drive
zip_path = "/content/drive/MyDrive/bart_paws_final_model.zip"
extract_path = "/content/bart_paws_final_model"

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print(f"Model unzipped to: {extract_path}")

# Load the fine-tuned BART model
model_name = extract_path
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

# Upload CSV File
print("Upload your dataset (CSV file)...")
uploaded = files.upload()

# Get the uploaded file name
file_name = list(uploaded.keys())[0]
print(f"Dataset uploaded: {file_name}")

# Load dataset
df = pd.read_csv(file_name)

# Ensure the dataset contains the expected column
if 'sentence' not in df.columns:
    raise ValueError("Dataset must contain a column named 'sentence'")

# Function to generate paraphrases using fine-tuned BART
def paraphrase(text):
    """Paraphrase input text using Fine-Tuned BART."""
    encoding = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **encoding,
            max_length=512,
            num_return_sequences=1,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            repetition_penalty=2.0
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Apply paraphrasing to the dataset
tqdm.pandas()
df["paraphrased_sentence"] = df["sentence"].progress_apply(paraphrase)

# Compute similarity using TF-IDF + Cosine Similarity
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["sentence"].tolist() + df["paraphrased_sentence"].tolist())

# Extracting the similarity scores
original_vectors = tfidf_matrix[:len(df)]
paraphrased_vectors = tfidf_matrix[len(df):]

similarities = [cosine_similarity(original_vectors[i], paraphrased_vectors[i])[0][0] for i in range(len(df))]

# Add similarity scores to DataFrame
df["similarity_score"] = similarities

# Calculate mean and standard deviation of similarity scores
mean_similarity = np.mean(similarities)
std_dev_similarity = np.std(similarities)

# Initialize ROUGE and BLEU score computation
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
bleu_scores = []
rouge_scores = {"rouge-1": [], "rouge-2": [], "rouge-l": []}

smoothing = SmoothingFunction().method1

for i in range(len(df)):
    ref = df.loc[i, "sentence"]
    pred = df.loc[i, "paraphrased_sentence"]

    # Compute BLEU score
    bleu = sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothing)
    bleu_scores.append(bleu)

    # Compute ROUGE scores
    rouge_score = scorer.score(ref, pred)
    rouge_scores["rouge-1"].append(rouge_score["rouge1"].fmeasure)
    rouge_scores["rouge-2"].append(rouge_score["rouge2"].fmeasure)
    rouge_scores["rouge-l"].append(rouge_score["rougeL"].fmeasure)

# Add BLEU and ROUGE scores to DataFrame
df["bleu_score"] = bleu_scores
df["rouge-1"] = rouge_scores["rouge-1"]
df["rouge-2"] = rouge_scores["rouge-2"]
df["rouge-l"] = rouge_scores["rouge-l"]

# Calculate mean and standard deviation for BLEU and ROUGE scores
mean_bleu = np.mean(bleu_scores)
std_bleu = np.std(bleu_scores)

mean_rouge_1 = np.mean(rouge_scores["rouge-1"])
std_rouge_1 = np.std(rouge_scores["rouge-1"])

mean_rouge_2 = np.mean(rouge_scores["rouge-2"])
std_rouge_2 = np.std(rouge_scores["rouge-2"])

mean_rouge_l = np.mean(rouge_scores["rouge-l"])
std_rouge_l = np.std(rouge_scores["rouge-l"])

# Save results
output_file = "PP-004.csv"
df.to_csv(output_file, index=False)

# Provide download link for output file
files.download(output_file)

# Print evaluation results
print(f"Results saved to {output_file}")
print(f"Mean Similarity Score: {mean_similarity:.4f}")
print(f"Standard Deviation of Similarity Scores: {std_dev_similarity:.4f}")
print(f"Mean BLEU Score: {mean_bleu:.4f}, Standard Deviation: {std_bleu:.4f}")
print(f"Mean ROUGE-1 Score: {mean_rouge_1:.4f}, Standard Deviation: {std_rouge_1:.4f}")
print(f"Mean ROUGE-2 Score: {mean_rouge_2:.4f}, Standard Deviation: {std_rouge_2:.4f}")
print(f"Mean ROUGE-L Score: {mean_rouge_l:.4f}, Standard Deviation: {std_rouge_l:.4f}")


Mounted at /content/drive
Model unzipped to: /content/bart_paws_final_model
Upload your dataset (CSV file)...


Saving paraphtrase_dataset.csv to paraphtrase_dataset.csv
Dataset uploaded: paraphtrase_dataset.csv


  0%|          | 0/480 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 480/480 [01:49<00:00,  4.39it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Results saved to PP-004.csv
Mean Similarity Score: 0.8558
Standard Deviation of Similarity Scores: 0.1291
Mean BLEU Score: 0.4098, Standard Deviation: 0.2258
Mean ROUGE-1 Score: 0.8972, Standard Deviation: 0.0903
Mean ROUGE-2 Score: 0.7482, Standard Deviation: 0.1999
Mean ROUGE-L Score: 0.8349, Standard Deviation: 0.1565
